In [34]:
####Estimation of time steps and number of layers needed for a given spectrum precision using either a dterministic or qdrift open quantum system simulation of the system
import openfermion as of

import sys
sys.path.append('./utils/')

from basis_utils import Sz,Sx,Sz, Sy
from basis_utils import read_spinach_info, build_list_ISTs, NormalizeBasis, build_symbolic_list_ISTs, MatRepLib
from simulation_utils import GenH0_Ham, HamMatRep, GenNOESYSpectrum, sqcosbell_2d_apod, GenFIDsignals
import scipy.io as spio
#from 
from scipy.linalg import expm
import cirq
import numpy as np
from matplotlib import pyplot as plt
from scipy.io import savemat



In [16]:
int(np.floor(10/3))

3

In [62]:
def GenFID_DetTrotsignals(Ham,List_jumps,T1,T2,rho0,coil,tmix,dt1,dt2,dtmix,Lx,Ly):
    """
    Implement a first-order deterministic Trotter simulation of the FID NOESY signals, to numerically explore optimization of number of Trotter steps needed in the simulation
    for a target precision
    Args:
    Ham: matrix representation of the "coherent" part of the Liouvillian siperoperator
    List_jumps: list of the matrix representation of the jump operators
    T1: maximal simulation time for the first phase of the protocol
    T2: maximal simulation time for the second phase of the protocol
    rho0: initial density matrix vector
    coil: vector that represents the obervable we trace over to compute FID
    tmix: total mixing time
    """

    #This is the convention for our Trotter step: apply first the coherent part of the super-Liouvillian operator, followed by the jump operators in the order they appear in the array...

    L_dt1 = expm(-1j*Ham*dt1)

    for i in range(len(List_jumps)):
        #print("Jump operator is",List_jumps[i])
        L_dt1 = expm(List_jumps[i]*dt1)@L_dt1

    R = np.copy(List_jumps[0])
    for i in range(len(List_jumps)-1):
        R+=List_jumps[i]

    print("Difference between Trotterized and exact L_dt1: ",np.linalg.norm(L_dt1-expm((-1j*Ham+R)*dt1)))

    L_dt2 = expm(-1j*Ham*dt2)

    for i in range(len(List_jumps)):
        L_dt2 = expm(List_jumps[i]*dt2)@L_dt2

    print("Difference between Trotterized and exact L_dt2: ",np.linalg.norm(L_dt2-expm((-1j*Ham+R)*dt2)))

    #Dim = Ham.shape[0]
    #Lnet = Ham+1j*R 
    #L_dt1 = expm(-1j*Lnet*dt1)
    #L_dt2 = expm(-1j*Lnet*dt2)
    L_dtmix = expm(-1j*Ham*dtmix)

    for i in range(len(List_jumps)):
        L_dtmix=expm(List_jumps[i]*dtmix)@L_dtmix


    ###Number of Trotter steps for the simulation of the mixing free evolution...
    Ntmix = int(np.floor(tmix/dtmix))
    #Number of Trotter steps for simulation of T1
    Tpts1 = int(np.floor(T1/dt1))
    Tpts2 = int(np.floor(T2/dt2))

    print("Number of Trotter steps for simulation of T1",Tpts1)
    print("Number of Trotter steps for simulation of T2",Tpts2)
    print("Number of Trotter steps for simulation of tmix", Ntmix)
    
    pulse_mix = np.copy(L_dtmix)
    for i in range(Ntmix):
        pulse_mix = L_dtmix@pulse_mix

    print("Difference between Trotterized pulse_mix and reference pulse_mix:", np.linalg.norm(pulse_mix-expm((-1j*Ham+R)*dtmix)))


    pulse_90x = expm(-1j*Lx*np.pi/2)
    pulse_90y = expm(-1j*Ly*np.pi/2)
    pulse_90mx = expm(1j*Lx*np.pi/2)
    pulse_90my = expm(1j*Ly*np.pi/2)

    #First 90x pulse:
    rho_t = np.copy(rho0)
    rho_t = np.dot(pulse_90x,rho_t)

    rho_stack = []
    rho_stack.append(rho_t)

    rho_temp = np.copy(rho_t)
    for i in range(1,Tpts1):
        rho_temp = np.dot(L_dt1,rho_temp)
        rho_stack.append(rho_temp)


    rho_stack1_1 = []
    rho_stack1_2 = []
    rho_stack1_3 = []
    rho_stack1_4 = []

    for i in range(Tpts1):
        rho_stack1_1.append(pulse_90y@pulse_mix@pulse_90x@rho_stack[i])
        rho_stack1_2.append(pulse_90y@pulse_mix@pulse_90y@rho_stack[i])
        rho_stack1_3.append(pulse_90y@pulse_mix@pulse_90mx@rho_stack[i])
        rho_stack1_4.append(pulse_90y@pulse_mix@pulse_90my@rho_stack[i])


    fid_temp_1 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_2 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_3 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_4 = np.zeros([Tpts2,Tpts1],dtype=complex)

    for i in range(Tpts1):
        rho1 = rho_stack1_1[i]
        rho2 = rho_stack1_2[i]
        rho3 = rho_stack1_3[i]
        rho4 = rho_stack1_4[i]

        for j in range(Tpts2):
            fid_temp_1[j,i] = np.dot(coil,rho1)
            rho1 = L_dt2@rho1

            fid_temp_2[j,i] = np.dot(coil,rho2)
            rho2 = L_dt2@rho2

            fid_temp_3[j,i] = np.dot(coil,rho3)
            rho3 = L_dt2@rho3

            fid_temp_4[j,i] = np.dot(coil,rho4)
            rho4 = L_dt2@rho4
    
    return fid_temp_1, fid_temp_2, fid_temp_3, fid_temp_4

def GenNOESYSpectrum_fromFID(fid_temp_1, fid_temp_2, fid_temp_3, fid_temp_4,zerofill1,zerofill2,returnFID=True):

    #fid_temp_1, fid_temp_2, fid_temp_3, fid_temp_4 = GenFIDsignals(Ham,R,Tpts1,Tpts2,rho0,coil,tmix,dt1,dt2,Lx,Ly)
    
    fid_test_cos = fid_temp_1 - fid_temp_3
    fid_test_sin = fid_temp_2 - fid_temp_4

    fid_cos = sqcosbell_2d_apod(fid_test_cos)
    fid_sin = sqcosbell_2d_apod(fid_test_sin)

    f1_cos = np.real(np.fft.fftshift(np.fft.fft2(fid_cos,[zerofill2],[0]),[0]))
    f1_sin = np.real(np.fft.fftshift(np.fft.fft2(fid_sin,[zerofill2],[0]),[0]))


    f1_states = f1_cos-1j*f1_sin

    spectrum = np.fft.fftshift(np.fft.fft2(f1_states,[zerofill1],[1]),[1])
    if returnFID:
        ###NOTE: return the FID witouth post-processing
        return spectrum, fid_test_cos-1j*fid_test_sin
    else:
        return spectrum



# The DFG case

In [5]:
text="""1      (0,0)   (0,0)   
  2      (0,0)   (1,1)   
  3      (0,0)   (1,0)   
  4      (0,0)   (1,-1)  
  5      (1,1)   (0,0)   
  6      (1,1)   (1,1)   
  7      (1,1)   (1,0)   
  8      (1,1)   (1,-1)  
  9      (1,0)   (0,0)   
  10     (1,0)   (1,1)   
  11     (1,0)   (1,0)   
  12     (1,0)   (1,-1)  
  13     (1,-1)  (0,0)   
  14     (1,-1)  (1,1)   
  15     (1,-1)  (1,0)   
  16     (1,-1)  (1,-1)  
"""

data = read_spinach_info(text)

basis = build_list_ISTs(data)
prefacts,Symb_basis = build_symbolic_list_ISTs(data)

#Normbasis = NormalizeBasis(basis,n_qubits=4,checkOrth=True) I have verified the orthonormalization of the basis
Normbasis = NormalizeBasis(basis,n_qubits=2,checkOrth=True)

In [6]:
###For the direct simulation approach we consider two-approaches: deterministic Trotter formula, and QDrift...

#We can perform the simulation in the Spinach basis, using the tools we already have access to...
offset = -46681
B0 = 9.3933
zeeman_scalar_1 = -113.8796
zeeman_scalar_2 = -129.8002
zeeman_scalars = [zeeman_scalar_1,zeeman_scalar_2]
Jcoup = 238.0633
Jcoups = np.zeros([2,2])
Jcoups[0,1] = Jcoup
gammaF = 251814800
#GenH0_Ham(offset,B0,zeeman_scalars,Jcoups,gammaF)

#freqs = [-2996.74,2996.74] # we were missing a factor of 2!
#freqs = [0.0,2*2996.74]
#freqs = [-11968.88/(2*np.pi),6860.19/(2*np.pi)]

#H0_Ham_dfg = GenH0_Ham_allparam(freqs,Jcoups,2)

#zeeman_scalars = [0.0,-15.920]
#zeeman_scalars = [0.0, -15.920599]
#offset=0.0

H0_dfg_ref = GenH0_Ham(offset,B0,zeeman_scalars,Jcoups,gammaF)


#H0_dfg_frame =  HamMatRep(H0_Ham_dfg,Normbasis,n_qubits=2)

#H0_dfg_frame =  HamMatRep(H0_Ham_dfg,Normbasis)
H0_dfg_mat =  HamMatRep(H0_dfg_ref,Normbasis)



In [10]:
###List of jump operators in their matrix representation...
List_jumps=[]

List_jumps.append(4.38*MatRepLib(Normbasis,Sz(0)*Sz(1),Sz(0)*Sz(1),n_qubits=2))

In [85]:
###Simulation to get FID signals...
loadMat = spio.loadmat('./data/DFG.mat',squeeze_me=True)
AuxMats = spio.loadmat('./data/DFG_NOESYmatrices.mat',squeeze_me=True)

###These are the parameters generated by Spinach
Ham = loadMat['p']['H'].item()
R = loadMat['p']['R'].item() # We can substitute this by any approximation to the relaxation matrix
t_grid1 = loadMat['p']['time_grid1'].item()
t_grid2 = loadMat['p']['time_grid2'].item()

###Dynamical evolution for calculation of 2D spectra...
##TODO: we can simply modify the script to incorporate 1) retrieval of synthesized circuits,
#and 2) the already-developed circuit simulator

Tpts1 = len(t_grid1)
Tpts2 = len(t_grid2)

Dim = Ham.shape[0]

rho0 = np.array(AuxMats['rho0'].toarray())
rho0 = rho0.flatten()
coil = AuxMats['coil']


tmix = 0.5
dt1 = 1.1561e-04
dt2 = 1.1561e-04

##Parameters for Fourier transform
zerofill1 = 1024
zerofill2 = 1024

##Definition of pulses in the experiment...
#This also depends on the definition of the basis....
Lx = AuxMats['Lx'].toarray()
Ly = AuxMats['Ly'].toarray()



#GenFID_DetTrotsignals(H0_dfg_mat,List_jumps,T1,T2,rho0,coil,tmix,dt1,dt2,dtmix,Lx,Ly)
fid_1_ref, fid_2_ref, fid_3_ref, fid_4_ref =  GenFIDsignals(Ham,List_jumps[0],Tpts1,Tpts2,rho0,coil,tmix,dt1,dt2,Lx,Ly)
#GenNOESYSpectrum(Ham,R,Tpts1,Tpts2,rho0,coil,tmix,dt1,dt2,zerofill1,zerofill2,Lx,Ly,returnFID=False)

In [91]:
T1 = Tpts1*dt1
T2 = Tpts2*dt2
dtmix = 1e-5
fid_temp_1, fid_temp_2, fid_temp_3, fid_temp_4 = GenFID_DetTrotsignals(H0_dfg_mat,List_jumps,T1,T2,rho0,coil,tmix,dt1,dt2,dtmix,Lx,Ly)

Difference between Trotterized and exact L_dt1:  1.1863596207543848e-15
Difference between Trotterized and exact L_dt2:  1.1863596207543848e-15
Number of Trotter steps for simulation of T1 512
Number of Trotter steps for simulation of T2 1024
Number of Trotter steps for simulation of tmix 49999
Difference between Trotterized pulse_mix and reference pulse_mix: 4.983979294968198


In [92]:
#Saving results for plotting in Matlab...

savemat('Ref_truncR_FID_DFG_cos.mat', {'FID_cos': fid_1_ref-fid_3_ref})
savemat('Ref_truncR_FID_DFG_sin.mat', {'FID_sin': fid_2_ref-fid_4_ref})

savemat('Trot_truncR_FID_DFG_cos.mat', {'FID_cos': fid_temp_1-fid_temp_3})
savemat('Trot_truncR_FID_DFG_sin.mat', {'FID_sin': fid_temp_2-fid_temp_4})



# The Alanine case

In [93]:
text4="""1      (0,0)   (0,0)   (0,0)   (0,0)   
  2      (0,0)   (0,0)   (0,0)   (1,1)   
  3      (0,0)   (0,0)   (0,0)   (1,0)   
  4      (0,0)   (0,0)   (0,0)   (1,-1)  
  5      (0,0)   (0,0)   (1,1)   (0,0)   
  6      (0,0)   (0,0)   (1,1)   (1,1)   
  7      (0,0)   (0,0)   (1,1)   (1,0)   
  8      (0,0)   (0,0)   (1,1)   (1,-1)  
  9      (0,0)   (0,0)   (1,0)   (0,0)   
  10     (0,0)   (0,0)   (1,0)   (1,1)   
  11     (0,0)   (0,0)   (1,0)   (1,0)   
  12     (0,0)   (0,0)   (1,0)   (1,-1)  
  13     (0,0)   (0,0)   (1,-1)  (0,0)   
  14     (0,0)   (0,0)   (1,-1)  (1,1)   
  15     (0,0)   (0,0)   (1,-1)  (1,0)   
  16     (0,0)   (0,0)   (1,-1)  (1,-1)  
  17     (0,0)   (1,1)   (0,0)   (0,0)   
  18     (0,0)   (1,1)   (0,0)   (1,1)   
  19     (0,0)   (1,1)   (0,0)   (1,0)   
  20     (0,0)   (1,1)   (0,0)   (1,-1)  
  21     (0,0)   (1,1)   (1,1)   (0,0)   
  22     (0,0)   (1,1)   (1,1)   (1,1)   
  23     (0,0)   (1,1)   (1,1)   (1,0)   
  24     (0,0)   (1,1)   (1,1)   (1,-1)  
  25     (0,0)   (1,1)   (1,0)   (0,0)   
  26     (0,0)   (1,1)   (1,0)   (1,1)   
  27     (0,0)   (1,1)   (1,0)   (1,0)   
  28     (0,0)   (1,1)   (1,0)   (1,-1)  
  29     (0,0)   (1,1)   (1,-1)  (0,0)   
  30     (0,0)   (1,1)   (1,-1)  (1,1)   
  31     (0,0)   (1,1)   (1,-1)  (1,0)   
  32     (0,0)   (1,1)   (1,-1)  (1,-1)  
  33     (0,0)   (1,0)   (0,0)   (0,0)   
  34     (0,0)   (1,0)   (0,0)   (1,1)   
  35     (0,0)   (1,0)   (0,0)   (1,0)   
  36     (0,0)   (1,0)   (0,0)   (1,-1)  
  37     (0,0)   (1,0)   (1,1)   (0,0)   
  38     (0,0)   (1,0)   (1,1)   (1,1)   
  39     (0,0)   (1,0)   (1,1)   (1,0)   
  40     (0,0)   (1,0)   (1,1)   (1,-1)  
  41     (0,0)   (1,0)   (1,0)   (0,0)   
  42     (0,0)   (1,0)   (1,0)   (1,1)   
  43     (0,0)   (1,0)   (1,0)   (1,0)   
  44     (0,0)   (1,0)   (1,0)   (1,-1)  
  45     (0,0)   (1,0)   (1,-1)  (0,0)   
  46     (0,0)   (1,0)   (1,-1)  (1,1)   
  47     (0,0)   (1,0)   (1,-1)  (1,0)   
  48     (0,0)   (1,0)   (1,-1)  (1,-1)  
  49     (0,0)   (1,-1)  (0,0)   (0,0)   
  50     (0,0)   (1,-1)  (0,0)   (1,1)   
  51     (0,0)   (1,-1)  (0,0)   (1,0)   
  52     (0,0)   (1,-1)  (0,0)   (1,-1)  
  53     (0,0)   (1,-1)  (1,1)   (0,0)   
  54     (0,0)   (1,-1)  (1,1)   (1,1)   
  55     (0,0)   (1,-1)  (1,1)   (1,0)   
  56     (0,0)   (1,-1)  (1,1)   (1,-1)  
  57     (0,0)   (1,-1)  (1,0)   (0,0)   
  58     (0,0)   (1,-1)  (1,0)   (1,1)   
  59     (0,0)   (1,-1)  (1,0)   (1,0)   
  60     (0,0)   (1,-1)  (1,0)   (1,-1)  
  61     (0,0)   (1,-1)  (1,-1)  (0,0)   
  62     (0,0)   (1,-1)  (1,-1)  (1,1)   
  63     (0,0)   (1,-1)  (1,-1)  (1,0)   
  64     (0,0)   (1,-1)  (1,-1)  (1,-1)  
  65     (1,1)   (0,0)   (0,0)   (0,0)   
  66     (1,1)   (0,0)   (0,0)   (1,1)   
  67     (1,1)   (0,0)   (0,0)   (1,0)   
  68     (1,1)   (0,0)   (0,0)   (1,-1)  
  69     (1,1)   (0,0)   (1,1)   (0,0)   
  70     (1,1)   (0,0)   (1,1)   (1,1)   
  71     (1,1)   (0,0)   (1,1)   (1,0)   
  72     (1,1)   (0,0)   (1,1)   (1,-1)  
  73     (1,1)   (0,0)   (1,0)   (0,0)   
  74     (1,1)   (0,0)   (1,0)   (1,1)   
  75     (1,1)   (0,0)   (1,0)   (1,0)   
  76     (1,1)   (0,0)   (1,0)   (1,-1)  
  77     (1,1)   (0,0)   (1,-1)  (0,0)   
  78     (1,1)   (0,0)   (1,-1)  (1,1)   
  79     (1,1)   (0,0)   (1,-1)  (1,0)   
  80     (1,1)   (0,0)   (1,-1)  (1,-1)  
  81     (1,1)   (1,1)   (0,0)   (0,0)   
  82     (1,1)   (1,1)   (0,0)   (1,1)   
  83     (1,1)   (1,1)   (0,0)   (1,0)   
  84     (1,1)   (1,1)   (0,0)   (1,-1)  
  85     (1,1)   (1,1)   (1,1)   (0,0)   
  86     (1,1)   (1,1)   (1,1)   (1,1)   
  87     (1,1)   (1,1)   (1,1)   (1,0)   
  88     (1,1)   (1,1)   (1,1)   (1,-1)  
  89     (1,1)   (1,1)   (1,0)   (0,0)   
  90     (1,1)   (1,1)   (1,0)   (1,1)   
  91     (1,1)   (1,1)   (1,0)   (1,0)   
  92     (1,1)   (1,1)   (1,0)   (1,-1)  
  93     (1,1)   (1,1)   (1,-1)  (0,0)   
  94     (1,1)   (1,1)   (1,-1)  (1,1)   
  95     (1,1)   (1,1)   (1,-1)  (1,0)   
  96     (1,1)   (1,1)   (1,-1)  (1,-1)  
  97     (1,1)   (1,0)   (0,0)   (0,0)   
  98     (1,1)   (1,0)   (0,0)   (1,1)   
  99     (1,1)   (1,0)   (0,0)   (1,0)   
  100    (1,1)   (1,0)   (0,0)   (1,-1)  
  101    (1,1)   (1,0)   (1,1)   (0,0)   
  102    (1,1)   (1,0)   (1,1)   (1,1)   
  103    (1,1)   (1,0)   (1,1)   (1,0)   
  104    (1,1)   (1,0)   (1,1)   (1,-1)  
  105    (1,1)   (1,0)   (1,0)   (0,0)   
  106    (1,1)   (1,0)   (1,0)   (1,1)   
  107    (1,1)   (1,0)   (1,0)   (1,0)   
  108    (1,1)   (1,0)   (1,0)   (1,-1)  
  109    (1,1)   (1,0)   (1,-1)  (0,0)   
  110    (1,1)   (1,0)   (1,-1)  (1,1)   
  111    (1,1)   (1,0)   (1,-1)  (1,0)   
  112    (1,1)   (1,0)   (1,-1)  (1,-1)  
  113    (1,1)   (1,-1)  (0,0)   (0,0)   
  114    (1,1)   (1,-1)  (0,0)   (1,1)   
  115    (1,1)   (1,-1)  (0,0)   (1,0)   
  116    (1,1)   (1,-1)  (0,0)   (1,-1)  
  117    (1,1)   (1,-1)  (1,1)   (0,0)   
  118    (1,1)   (1,-1)  (1,1)   (1,1)   
  119    (1,1)   (1,-1)  (1,1)   (1,0)   
  120    (1,1)   (1,-1)  (1,1)   (1,-1)  
  121    (1,1)   (1,-1)  (1,0)   (0,0)   
  122    (1,1)   (1,-1)  (1,0)   (1,1)   
  123    (1,1)   (1,-1)  (1,0)   (1,0)   
  124    (1,1)   (1,-1)  (1,0)   (1,-1)  
  125    (1,1)   (1,-1)  (1,-1)  (0,0)   
  126    (1,1)   (1,-1)  (1,-1)  (1,1)   
  127    (1,1)   (1,-1)  (1,-1)  (1,0)   
  128    (1,1)   (1,-1)  (1,-1)  (1,-1)  
  129    (1,0)   (0,0)   (0,0)   (0,0)   
  130    (1,0)   (0,0)   (0,0)   (1,1)   
  131    (1,0)   (0,0)   (0,0)   (1,0)   
  132    (1,0)   (0,0)   (0,0)   (1,-1)  
  133    (1,0)   (0,0)   (1,1)   (0,0)   
  134    (1,0)   (0,0)   (1,1)   (1,1)   
  135    (1,0)   (0,0)   (1,1)   (1,0)   
  136    (1,0)   (0,0)   (1,1)   (1,-1)  
  137    (1,0)   (0,0)   (1,0)   (0,0)   
  138    (1,0)   (0,0)   (1,0)   (1,1)   
  139    (1,0)   (0,0)   (1,0)   (1,0)   
  140    (1,0)   (0,0)   (1,0)   (1,-1)  
  141    (1,0)   (0,0)   (1,-1)  (0,0)   
  142    (1,0)   (0,0)   (1,-1)  (1,1)   
  143    (1,0)   (0,0)   (1,-1)  (1,0)   
  144    (1,0)   (0,0)   (1,-1)  (1,-1)  
  145    (1,0)   (1,1)   (0,0)   (0,0)   
  146    (1,0)   (1,1)   (0,0)   (1,1)   
  147    (1,0)   (1,1)   (0,0)   (1,0)   
  148    (1,0)   (1,1)   (0,0)   (1,-1)  
  149    (1,0)   (1,1)   (1,1)   (0,0)   
  150    (1,0)   (1,1)   (1,1)   (1,1)   
  151    (1,0)   (1,1)   (1,1)   (1,0)   
  152    (1,0)   (1,1)   (1,1)   (1,-1)  
  153    (1,0)   (1,1)   (1,0)   (0,0)   
  154    (1,0)   (1,1)   (1,0)   (1,1)   
  155    (1,0)   (1,1)   (1,0)   (1,0)   
  156    (1,0)   (1,1)   (1,0)   (1,-1)  
  157    (1,0)   (1,1)   (1,-1)  (0,0)   
  158    (1,0)   (1,1)   (1,-1)  (1,1)   
  159    (1,0)   (1,1)   (1,-1)  (1,0)   
  160    (1,0)   (1,1)   (1,-1)  (1,-1)  
  161    (1,0)   (1,0)   (0,0)   (0,0)   
  162    (1,0)   (1,0)   (0,0)   (1,1)   
  163    (1,0)   (1,0)   (0,0)   (1,0)   
  164    (1,0)   (1,0)   (0,0)   (1,-1)  
  165    (1,0)   (1,0)   (1,1)   (0,0)   
  166    (1,0)   (1,0)   (1,1)   (1,1)   
  167    (1,0)   (1,0)   (1,1)   (1,0)   
  168    (1,0)   (1,0)   (1,1)   (1,-1)  
  169    (1,0)   (1,0)   (1,0)   (0,0)   
  170    (1,0)   (1,0)   (1,0)   (1,1)   
  171    (1,0)   (1,0)   (1,0)   (1,0)   
  172    (1,0)   (1,0)   (1,0)   (1,-1)  
  173    (1,0)   (1,0)   (1,-1)  (0,0)   
  174    (1,0)   (1,0)   (1,-1)  (1,1)   
  175    (1,0)   (1,0)   (1,-1)  (1,0)   
  176    (1,0)   (1,0)   (1,-1)  (1,-1)  
  177    (1,0)   (1,-1)  (0,0)   (0,0)   
  178    (1,0)   (1,-1)  (0,0)   (1,1)   
  179    (1,0)   (1,-1)  (0,0)   (1,0)   
  180    (1,0)   (1,-1)  (0,0)   (1,-1)  
  181    (1,0)   (1,-1)  (1,1)   (0,0)   
  182    (1,0)   (1,-1)  (1,1)   (1,1)   
  183    (1,0)   (1,-1)  (1,1)   (1,0)   
  184    (1,0)   (1,-1)  (1,1)   (1,-1)  
  185    (1,0)   (1,-1)  (1,0)   (0,0)   
  186    (1,0)   (1,-1)  (1,0)   (1,1)   
  187    (1,0)   (1,-1)  (1,0)   (1,0)   
  188    (1,0)   (1,-1)  (1,0)   (1,-1)  
  189    (1,0)   (1,-1)  (1,-1)  (0,0)   
  190    (1,0)   (1,-1)  (1,-1)  (1,1)   
  191    (1,0)   (1,-1)  (1,-1)  (1,0)   
  192    (1,0)   (1,-1)  (1,-1)  (1,-1)  
  193    (1,-1)  (0,0)   (0,0)   (0,0)   
  194    (1,-1)  (0,0)   (0,0)   (1,1)   
  195    (1,-1)  (0,0)   (0,0)   (1,0)   
  196    (1,-1)  (0,0)   (0,0)   (1,-1)  
  197    (1,-1)  (0,0)   (1,1)   (0,0)   
  198    (1,-1)  (0,0)   (1,1)   (1,1)   
  199    (1,-1)  (0,0)   (1,1)   (1,0)   
  200    (1,-1)  (0,0)   (1,1)   (1,-1)  
  201    (1,-1)  (0,0)   (1,0)   (0,0)   
  202    (1,-1)  (0,0)   (1,0)   (1,1)   
  203    (1,-1)  (0,0)   (1,0)   (1,0)   
  204    (1,-1)  (0,0)   (1,0)   (1,-1)  
  205    (1,-1)  (0,0)   (1,-1)  (0,0)   
  206    (1,-1)  (0,0)   (1,-1)  (1,1)   
  207    (1,-1)  (0,0)   (1,-1)  (1,0)   
  208    (1,-1)  (0,0)   (1,-1)  (1,-1)  
  209    (1,-1)  (1,1)   (0,0)   (0,0)   
  210    (1,-1)  (1,1)   (0,0)   (1,1)   
  211    (1,-1)  (1,1)   (0,0)   (1,0)   
  212    (1,-1)  (1,1)   (0,0)   (1,-1)  
  213    (1,-1)  (1,1)   (1,1)   (0,0)   
  214    (1,-1)  (1,1)   (1,1)   (1,1)   
  215    (1,-1)  (1,1)   (1,1)   (1,0)   
  216    (1,-1)  (1,1)   (1,1)   (1,-1)  
  217    (1,-1)  (1,1)   (1,0)   (0,0)   
  218    (1,-1)  (1,1)   (1,0)   (1,1)   
  219    (1,-1)  (1,1)   (1,0)   (1,0)   
  220    (1,-1)  (1,1)   (1,0)   (1,-1)  
  221    (1,-1)  (1,1)   (1,-1)  (0,0)   
  222    (1,-1)  (1,1)   (1,-1)  (1,1)   
  223    (1,-1)  (1,1)   (1,-1)  (1,0)   
  224    (1,-1)  (1,1)   (1,-1)  (1,-1)  
  225    (1,-1)  (1,0)   (0,0)   (0,0)   
  226    (1,-1)  (1,0)   (0,0)   (1,1)   
  227    (1,-1)  (1,0)   (0,0)   (1,0)   
  228    (1,-1)  (1,0)   (0,0)   (1,-1)  
  229    (1,-1)  (1,0)   (1,1)   (0,0)   
  230    (1,-1)  (1,0)   (1,1)   (1,1)   
  231    (1,-1)  (1,0)   (1,1)   (1,0)   
  232    (1,-1)  (1,0)   (1,1)   (1,-1)  
  233    (1,-1)  (1,0)   (1,0)   (0,0)   
  234    (1,-1)  (1,0)   (1,0)   (1,1)   
  235    (1,-1)  (1,0)   (1,0)   (1,0)   
  236    (1,-1)  (1,0)   (1,0)   (1,-1)  
  237    (1,-1)  (1,0)   (1,-1)  (0,0)   
  238    (1,-1)  (1,0)   (1,-1)  (1,1)   
  239    (1,-1)  (1,0)   (1,-1)  (1,0)   
  240    (1,-1)  (1,0)   (1,-1)  (1,-1)  
  241    (1,-1)  (1,-1)  (0,0)   (0,0)   
  242    (1,-1)  (1,-1)  (0,0)   (1,1)   
  243    (1,-1)  (1,-1)  (0,0)   (1,0)   
  244    (1,-1)  (1,-1)  (0,0)   (1,-1)  
  245    (1,-1)  (1,-1)  (1,1)   (0,0)   
  246    (1,-1)  (1,-1)  (1,1)   (1,1)   
  247    (1,-1)  (1,-1)  (1,1)   (1,0)   
  248    (1,-1)  (1,-1)  (1,1)   (1,-1)  
  249    (1,-1)  (1,-1)  (1,0)   (0,0)   
  250    (1,-1)  (1,-1)  (1,0)   (1,1)   
  251    (1,-1)  (1,-1)  (1,0)   (1,0)   
  252    (1,-1)  (1,-1)  (1,0)   (1,-1)  
  253    (1,-1)  (1,-1)  (1,-1)  (0,0)   
  254    (1,-1)  (1,-1)  (1,-1)  (1,1)   
  255    (1,-1)  (1,-1)  (1,-1)  (1,0)   
  256    (1,-1)  (1,-1)  (1,-1)  (1,-1)"""



In [94]:
data_ala = read_spinach_info(text4)

basis_ala = build_list_ISTs(data_ala)
prefacts,Symb_ALA_basis = build_symbolic_list_ISTs(data_ala)

#Normbasis = NormalizeBasis(basis,n_qubits=4,checkOrth=True) I have verified the orthonormalization of the basis
Normbasis_ala = NormalizeBasis(basis_ala,n_qubits=4,checkOrth=False)
Normbasis_ala = np.array(Normbasis_ala)

In [98]:
offset = 1500
B0 = 14.1
zeeman_scalar_1 = 3.4938
zeeman_scalar_2 = 1.7947
zeeman_scalar_3 = 1.7947
zeeman_scalar_4 = 1.7947
zeeman_scalars = [zeeman_scalar_1,zeeman_scalar_2,zeeman_scalar_3,zeeman_scalar_4]


#Jcoup = 238.0633

Jcoups = np.zeros([4,4])
#First, in Hz...
Jcoups[0,1] = 3.82
Jcoups[0,2] = 3.82
Jcoups[0,3] = 3.82
Jcoups[1,2] = -7.0
Jcoups[1,3] = -7.0
Jcoups[2,3] = -7.0

gammaH = 2.6752e8


H0_ala_ref = GenH0_Ham(offset,B0,zeeman_scalars,Jcoups,gammaH)


#H0_dfg_frame =  HamMatRep(H0_Ham_dfg,Normbasis,n_qubits=2)

#H0_dfg_frame =  HamMatRep(H0_Ham_dfg,Normbasis)
#H0_ala_mat =  HamMatRep(H0_ala_ref,Normbasis_ala,n_qubits=4)


####Construction of the jump operators...

#List_ala_jumps = []

#List_ala_jumps.append(1.44*MatRepLib(Normbasis_ala,Sz(0)*Sz(1),Sz(0)*Sz(1),n_qubits=4))
#List_ala_jumps.append(1.39*MatRepLib(Normbasis_ala,Sz(1)*Sz(2),Sz(1)*Sz(2),n_qubits=4))
#List_ala_jumps.append(1.37*MatRepLib(Normbasis_ala,Sz(0)*Sz(2),Sz(0)*Sz(2),n_qubits=4))





In [105]:
#####Saving the list of jump operators....
#import pickle

#Dict_jump = {'JOps':List_ala_jumps}

#with open('./data/ALA_JOps.pk', 'wb') as handle:
#    pickle.dump(Dict_jump, handle)


In [187]:
Tpts1*0.25e-3

0.256

In [188]:
0.256/0.00512

50.0

In [189]:
loadMat = spio.loadmat('./data/NOESYdata_ALA_withGradients.mat',squeeze_me=True)

t_grid1 = loadMat['p']['time_grid1'].item()
t_grid2 = loadMat['p']['time_grid2'].item()

R_ala = loadMat['p']['R'].item()
H_ala = loadMat['p']['H'].item().toarray()

rho0 = np.array(loadMat['p']['rho0'].item().toarray())

coil = np.array(loadMat['p']['coil'].item())

Lx = loadMat['p']['Lx'].item().toarray() 
Ly = loadMat['p']['Ly'].item().toarray() 

###Dynamical evolution for calculation of 2D spectra...
#Tpts1 = len(t_grid1)
#Tpts2 = len(t_grid2)

#Parameters taken from Spinach script
tmix = 1.0
#dt1 = 0.25e-3
#dt2 = 0.25e-3
dt1 = 0.00512
dt2 = 0.00512
Tpts1 = 50
Tpts2 = 50

##Parameters for Fourier transform
zerofill1 = 4096
zerofill2 = 4096

#Defining the truncated version of the Relaxation matrix...
R_trunc = np.copy(List_ala_jumps[0])

R_trunc += List_ala_jumps[1]
R_trunc += List_ala_jumps[2]



#R_trunc is going to be the sum of the matrix representation of the three jump operators
fid_1_ref, fid_2_ref, fid_3_ref, fid_4_ref =  GenFIDsignals(H_ala,R_trunc,Tpts1,Tpts2,rho0,coil,tmix,dt1,dt2,Lx,Ly)


In [205]:
test3= 27.21*np.array([[0.0027068883,-0.0019635208],[-0.0019635208,0.0027068883]])

In [206]:
np.linalg.eigvals(test3)

array([0.12708183, 0.02022703])

In [209]:
(0.12708183-0.02022703)/27.21

0.003927041528849687

In [191]:
test = 27.21*np.array([[0.0026801601,-0.0019902009],[-0.0019902009,0.0026801601]])

In [204]:
np.linalg.eigvals(test)

array([0.12708052, 0.01877379])

In [222]:
J12 = 27.21*(-0.0001385854-0.0000716364)
J23 = 27.21*(0.0000729342+0.0000118255)
J13 = 27.21*(0.0141751091+0.0148969739)

test2 = J12*of.get_sparse_operator(Sz(0)*Sz(1)+Sy(0)*Sy(1)+Sx(0)*Sx(1),n_qubits=3)
test2+= J23*of.get_sparse_operator(Sz(1)*Sz(2)+Sy(1)*Sy(2)+Sx(1)*Sx(2),n_qubits=3)
test2+= J13*of.get_sparse_operator(Sz(0)*Sz(2)+Sy(0)*Sy(2)+Sx(0)*Sx(2),n_qubits=3)

eigvals, eigvects =np.linalg.eig(test2.toarray())
#of.eigenspectrum(Sz(0)*Sz(1)+Sy(0)*Sy(1)+Sx(0)*Sx(1))

In [226]:
eigvals+(-0.52450534+0.59330377)

array([-0.52450534+0.j,  0.26570782+0.j,  0.26828342+0.j, -0.52450534+0.j,
        0.26828342+0.j,  0.26570782+0.j,  0.26570782+0.j,  0.26570782+0.j])

In [221]:
####The tri-radical state:

test_tri = 27.21*np.array([[0.0094728395,-0.0001385854,0.0000729342],
                     [-0.0000716364,-0.0052157853,0.0141751091],
                     [0.0000118255,0.0148969739,-0.0042570542]])

np.linalg.eigvals(test_tri)

array([-0.52450534,  0.25762079,  0.26688455])

In [214]:
eigvals

array([-2.57287566+0.j,  0.07287566+0.j,  1.25      +0.j, -2.57287566+0.j,
        1.25      +0.j,  0.07287566+0.j,  1.25      +0.j,  1.25      +0.j])

In [203]:
eigvects[:,1]

array([-0.        -0.j, -0.70710678-0.j,  0.70710678+0.j, -0.        -0.j])

In [195]:
eigvals

array([ 0.25+0.j, -0.75+0.j,  0.25+0.j,  0.25+0.j])

In [114]:
T1 = Tpts1*dt1
T2 = Tpts2*dt2
dtmix = 1e-3
fid_temp_1, fid_temp_2, fid_temp_3, fid_temp_4 = GenFID_DetTrotsignals(H_ala,List_ala_jumps,T1,T2,rho0,coil,tmix,dt1,dt2,dtmix,Lx,Ly)

Difference between Trotterized and exact L_dt1:  0.0004972498640126282
Difference between Trotterized and exact L_dt2:  0.0004972498640126282
Number of Trotter steps for simulation of T1 1024
Number of Trotter steps for simulation of T2 1024
Number of Trotter steps for simulation of tmix 1000
Difference between Trotterized pulse_mix and reference pulse_mix: 19.689629706283515


/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_8809/1472797926.py:105: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_temp_1[j,i] = np.dot(coil,rho1)
/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_8809/1472797926.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_temp_2[j,i] = np.dot(coil,rho2)
/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_8809/1472797926.py:111: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_temp_3

In [190]:
savemat('Ref_truncR_FID_ALA_cos.mat', {'FID_cos': fid_1_ref-fid_3_ref})
savemat('Ref_truncR_FID_ALA_sin.mat', {'FID_sin': fid_2_ref-fid_4_ref})

#savemat('Trot_truncR_FID_ALA_cos.mat', {'FID_cos': fid_temp_1-fid_temp_3})
#savemat('Trot_truncR_FID_ALA_sin.mat', {'FID_sin': fid_temp_2-fid_temp_4})

# QDrift implementation

In [174]:
# QDrift implementation.

def Samp_Gates(ListGates,List_weights,N):
    #Return a QDrift product formula that simulates a target time T using N samples drawn from the probability distribution...

    Gamma = np.sum(List_weights)
    prob_dist = (1.0/Gamma)*np.array(List_weights)

    #Draw a sample of size N for the implementation of the product formula...
    values = np.arange(len(ListGates))
    samples = np.random.choice(values, size=N, p=prob_dist)
    
    #print("Indixes of gates sampled:",samples)
    drawn_jumps = []
    for i in range(len(samples)):
        if samples[i]!=0:
            drawn_jumps.append(samples[i])

    print("Indices of jump operators drwan during construction of QDrift channel", drawn_jumps)

    ProdF = np.copy(ListGates[samples[0]])
    for i in range(1,N):
        ProdF=ListGates[samples[i]]@ProdF

    return ProdF


def Normalize_and_weightOps(ListOps):
    """
    ListOps contains the generators of evolution, either the representation of the coherent part of evolution or the jump operators 
    """
    List_weights=[]

    for i in range(len(ListOps)):
        List_weights.append(np.max(np.abs(np.linalg.eigvals(ListOps[i]))))

    #Normalize the operator according to the weights...
    Norm_ops =[]

    for i in range(len(ListOps)):
        Norm_ops.append(ListOps[i]/List_weights[i])

    return Norm_ops, List_weights


def QDrift(H0,List_jumps,T,N):


    Norm_ops, List_weights= Normalize_and_weightOps([H0]+List_jumps)

    ###Definition of gates...
    List_Gates = []

    #for i in range(len(List_jumps)):
    #    List_weights.append(rates[i])
    Gamma = np.sum(np.array(List_weights))

    deltaT = T*Gamma/N

    #List_Gates.append(expm(-1j*H0*deltaT))

    for i in range(len(Norm_ops)):
        if i==0:###I assume that the first element of the operators correspond to H0!!!
            List_Gates.append(expm(-1j*Norm_ops[i]*deltaT))
        else:
            List_Gates.append(expm(Norm_ops[i]*deltaT))
        

    ProdF = Samp_Gates(List_Gates,List_weights,N)

    return ProdF






    

In [175]:
###Generation of QDrift-based NOESY spectrum..

def GenFID_DetQDriftsignals(Ham,List_jumps,T1,T2,rho0,coil,tmix,dt1,dt2,Ntmix,Lx,Ly):
    """
    Implement a first-order deterministic Trotter simulation of the FID NOESY signals, to numerically explore optimization of number of Trotter steps needed in the simulation
    for a target precision
    Args:
    Ham: matrix representation of the "coherent" part of the Liouvillian siperoperator
    List_jumps: list of the matrix representation of the jump operators
    T1: maximal simulation time for the first phase of the protocol
    T2: maximal simulation time for the second phase of the protocol
    rho0: initial density matrix vector
    coil: vector that represents the obervable we trace over to compute FID
    Ntmix: number of samples to drawn in QDrift for the simulation of the system for tmix
    """

    #This is the convention for our Trotter step: apply first the coherent part of the super-Liouvillian operator, followed by the jump operators in the order they appear in the array...

    L_dt1 = expm(-1j*Ham*dt1)

    for i in range(len(List_jumps)):
        #print("Jump operator is",List_jumps[i])
        L_dt1 = expm(List_jumps[i]*dt1)@L_dt1

    R = np.copy(List_jumps[0])
    for i in range(len(List_jumps)-1):
        R+=List_jumps[i]

    print("Difference between Trotterized and exact L_dt1: ",np.linalg.norm(L_dt1-expm((-1j*Ham+R)*dt1)))

    L_dt2 = expm(-1j*Ham*dt2)

    for i in range(len(List_jumps)):
        L_dt2 = expm(List_jumps[i]*dt2)@L_dt2

    print("Difference between Trotterized and exact L_dt2: ",np.linalg.norm(L_dt2-expm((-1j*Ham+R)*dt2)))

    #Dim = Ham.shape[0]
    #Lnet = Ham+1j*R 
    #L_dt1 = expm(-1j*Lnet*dt1)
    #L_dt2 = expm(-1j*Lnet*dt2)
    #L_dtmix = expm(-1j*Ham*dtmix)

    #for i in range(len(List_jumps)):
    #    L_dtmix=expm(List_jumps[i]*dtmix)@L_dtmix


    ###Number of Trotter steps for the simulation of the mixing free evolution...
    #Ntmix = int(np.floor(tmix/dtmix))
    #Number of Trotter steps for simulation of T1
    Tpts1 = int(np.floor(T1/dt1))
    Tpts2 = int(np.floor(T2/dt2))

    print("Number of Trotter steps for deterministic simulation of T1",Tpts1)
    print("Number of Trotter steps for deterministic simulation of T2",Tpts2)
    print("Number of Trotter steps for QDrift simulation of tmix", Ntmix)
    
    pulse_mix = QDrift(Ham,List_jumps,tmix,Ntmix)
    #for i in range(Ntmix):
    #    pulse_mix = L_dtmix@pulse_mix

    print("Difference between Trotterized pulse_mix and reference pulse_mix:", np.linalg.norm(pulse_mix-expm((-1j*Ham+R)*tmix/Ntmix)))


    pulse_90x = expm(-1j*Lx*np.pi/2)
    pulse_90y = expm(-1j*Ly*np.pi/2)
    pulse_90mx = expm(1j*Lx*np.pi/2)
    pulse_90my = expm(1j*Ly*np.pi/2)

    #First 90x pulse:
    rho_t = np.copy(rho0)
    rho_t = np.dot(pulse_90x,rho_t)

    rho_stack = []
    rho_stack.append(rho_t)

    rho_temp = np.copy(rho_t)
    for i in range(1,Tpts1):
        rho_temp = np.dot(L_dt1,rho_temp)
        rho_stack.append(rho_temp)


    rho_stack1_1 = []
    rho_stack1_2 = []
    rho_stack1_3 = []
    rho_stack1_4 = []

    for i in range(Tpts1):
        rho_stack1_1.append(pulse_90y@pulse_mix@pulse_90x@rho_stack[i])
        rho_stack1_2.append(pulse_90y@pulse_mix@pulse_90y@rho_stack[i])
        rho_stack1_3.append(pulse_90y@pulse_mix@pulse_90mx@rho_stack[i])
        rho_stack1_4.append(pulse_90y@pulse_mix@pulse_90my@rho_stack[i])


    fid_temp_1 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_2 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_3 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_4 = np.zeros([Tpts2,Tpts1],dtype=complex)

    for i in range(Tpts1):
        rho1 = rho_stack1_1[i]
        rho2 = rho_stack1_2[i]
        rho3 = rho_stack1_3[i]
        rho4 = rho_stack1_4[i]

        for j in range(Tpts2):
            fid_temp_1[j,i] = np.dot(coil,rho1)
            rho1 = L_dt2@rho1

            fid_temp_2[j,i] = np.dot(coil,rho2)
            rho2 = L_dt2@rho2

            fid_temp_3[j,i] = np.dot(coil,rho3)
            rho3 = L_dt2@rho3

            fid_temp_4[j,i] = np.dot(coil,rho4)
            rho4 = L_dt2@rho4
    
    return fid_temp_1, fid_temp_2, fid_temp_3, fid_temp_4


In [186]:
dt1*Tpts1,dt2*Tpts2

(5.24288, 5.24288)

In [182]:
.256/50

0.00512

In [183]:
loadMat = spio.loadmat('./data/NOESYdata_ALA_withGradients.mat',squeeze_me=True)

t_grid1 = loadMat['p']['time_grid1'].item()
t_grid2 = loadMat['p']['time_grid2'].item()

R_ala = loadMat['p']['R'].item()
H_ala = loadMat['p']['H'].item().toarray()

rho0 = np.array(loadMat['p']['rho0'].item().toarray())

coil = np.array(loadMat['p']['coil'].item())

Lx = loadMat['p']['Lx'].item().toarray() 
Ly = loadMat['p']['Ly'].item().toarray() 

###Dynamical evolution for calculation of 2D spectra...
Tpts1 = len(t_grid1)
Tpts2 = len(t_grid2)

#Parameters taken from Spinach script
tmix = 1.0
#dt1 = 0.25e-3
dt1 = 0.00512
#dt2 = 0.25e-3
dt2 = 0.00512

##Parameters for Fourier transform
zerofill1 = 4096
zerofill2 = 4096

#Defining the truncated version of the Relaxation matrix...
#R_trunc = np.copy(List_ala_jumps[0])

#R_trunc += List_ala_jumps[1]
#R_trunc += List_ala_jumps[2]


#fid_temp_1, fid_temp_2, fid_temp_3, fid_temp_4=GenFID_DetQDriftsignals(H_ala,List_ala_jumps,T1,T2,rho0,coil,tmix,dt1,dt2,2000,Lx,Ly)
fid_temp_1, fid_temp_2, fid_temp_3, fid_temp_4=GenFID_DetQDriftsignals(H_ala,List_ala_jumps,T1,T2,rho0,coil,tmix,dt1,dt2,50,Lx,Ly)

Difference between Trotterized and exact L_dt1:  0.010718667393298666
Difference between Trotterized and exact L_dt2:  0.010718667393298666
Number of Trotter steps for deterministic simulation of T1 50
Number of Trotter steps for deterministic simulation of T2 50
Number of Trotter steps for QDrift simulation of tmix 50
Indices of jump operators drwan during construction of QDrift channel []
Difference between Trotterized pulse_mix and reference pulse_mix: 22.415751301267022


/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_8809/424463602.py:107: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_temp_1[j,i] = np.dot(coil,rho1)
/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_8809/424463602.py:110: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_temp_2[j,i] = np.dot(coil,rho2)
/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_8809/424463602.py:113: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_temp_3[j,

In [184]:
savemat('QDrift_truncR_FID_ALA_cos.mat', {'FID_cos': fid_temp_1-fid_temp_3})
savemat('QDrift_truncR_FID_ALA_sin.mat', {'FID_sin': fid_temp_2-fid_temp_4})

In [173]:
List_ala_jumps

[array([[0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
        ...,
        [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j]]),
 array([[0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
        ...,
        [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j]]),
 array([[0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
        ...,
        [0.+

In [139]:
####We consider weights equivalent to the spectral norm of each of the operators...

List_weights = []

List_weights.append(np.max(np.abs(np.linalg.eigvals(H_ala))))

for i in range(len(List_ala_jumps)):
    List_weights.append(np.max(np.abs(np.linalg.eigvals(List_ala_jumps[i]))))

###Renormalize the generators of evolution...
H0_renorm = H_ala/List_weights[0]

List_renorm_jumps = []

for i in range(3):
    List_renorm_jumps.append(List_ala_jumps[i]/List_weights[i+1])





#QDrift(H0_renorm,List_renorm_jumps,List_weights,0.25,10)



array([[ 1.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j, ...,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j],
       [ 0.        +0.00000000e+00j, -0.31198002+4.34526476e-01j,
         0.        +0.00000000e+00j, ...,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j],
       [ 0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.5564925 +9.77348966e-18j, ...,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j],
       ...,
       [ 0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j, ..., -0.07305135+5.50753289e-01j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j],
       [ 0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j, ...,  0.        +0.00000000e+00j,
         0.11078198-8.42101033e-02j

In [160]:
####Some numerical testing of the QDrift channel....
T = 1.0

np.linalg.norm(expm((-1j*H_ala+R_trunc)*T)-QDrift(H0_renorm,List_renorm_jumps,List_weights,T,100))


Indixes of gates sampled: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


4.681063691566985

In [163]:
np.linalg.norm(expm((-1j*H_ala)*T)-QDrift(H0_renorm,List_renorm_jumps,List_weights,T,100))

Indixes of gates sampled: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


2.9975486980419697

In [167]:
np.linalg.norm(expm(-1j*H_ala*T)@rho0-expm(-1j*H0_renorm*np.sum(List_weights)*T)@rho0)

6.985878157568836e-13

In [166]:
np.sum(List_weights)/List_weights[0]

1.0000447965989938

In [161]:
np.linalg.norm(expm((-1j*H_ala+R_trunc)*T)-expm(-1j*H_ala*T))

3.8675275638659836

In [162]:
List_weights

[11719.639700179749,
 0.1800000000000003,
 0.1737500000000001,
 0.17125000000000018]

In [147]:
List_weights

[11719.639700179749,
 0.1800000000000003,
 0.1737500000000001,
 0.17125000000000018]

In [142]:
np.linalg.norm(expm((-1j*H_ala+R_trunc)*T))

14.993760773198927

In [125]:
np.max(np.abs(np.linalg.eigvals(List_ala_jumps[0])))

0.1800000000000003